In [1]:
import re
import os
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook, load_workbook

# Caminho da planilha
caminho_arquivo = r"C:\Users\Josenir\Desktop\codigos vsc\jogadores.xlsx"

# URL do jogador (troque aqui para outro jogador)
url = "https://www.transfermarkt.com.br/vinicius-junior/profil/spieler/371998"
player_id = url.split("/")[-1].split(".")[0]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.raise_for_status()
soup = BeautifulSoup(response.content, "html.parser")

# Coleta dos dados
name_span = soup.select_one('span.info-table__content--bold')
player_full_name = name_span.text.strip() if name_span else "Nome completo não encontrado"

player_name = soup.select_one('h1[class="data-header__headline-wrapper"]').text.strip().split('\n')[-1].strip()
shirt_number = soup.select_one('span[class="data-header__shirt-number"]').text.strip()

# Posição
position_li = soup.find('li', string=lambda text: text and "Posição" in text)
if not position_li:
    all_li = soup.find_all('li', class_='data-header__label')
    for li in all_li:
        if "Posição" in li.text:
            position_span = li.find('span', class_='data-header__content')
            position = position_span.text.strip() if position_span else "Posição não encontrada"
            break
    else:
        position = "Posição não encontrada"
else:
    position_span = position_li.find('span', class_='data-header__content')
    position = position_span.text.strip() if position_span else "Posição não encontrada"

# Nascimento
birth_span = soup.select_one('span[itemprop="birthDate"].data-header__content')
nascimento_idade = birth_span.text.strip() if birth_span else "Nascimento/Idade não encontrado"

# Altura
altura_span = soup.select_one('span[itemprop="height"].data-header__content')
altura = altura_span.text.strip() if altura_span else "Altura não encontrada"

# Nacionalidade
nacionalidade_span = soup.select_one('span[itemprop="nationality"].data-header__content')
nacionalidade = nacionalidade_span.text.strip() if nacionalidade_span else "Nacionalidade não encontrada"

# Pé dominante
foot_span = soup.find_all('span', class_='info-table__content info-table__content--bold')
dominant_foot = "Pé não encontrado"
for span in foot_span:
    if span.text.strip().lower() in ['direito', 'esquerdo', 'ambos']:
        dominant_foot = span.text.strip().capitalize()
        break

# Valor de mercado
market_tag = soup.select_one('a.data-header__market-value-wrapper')
if market_tag:
    raw_market = market_tag.text.strip()
    numero_str = re.search(r'[\d,.]+', raw_market)
    valor = float(numero_str.group().replace(',', '')) if numero_str else 0.0
    if 'mi' in raw_market.lower():
        valor *= 1_000_000
    elif 'k' in raw_market.lower():
        valor *= 1_000
    valor_em_milhoes = valor / 1_000_000
    market_value = f"€{valor_em_milhoes:.2f} mi"
else:
    market_value = "Valor de mercado não encontrado"

# Empresário
empresario_tag = soup.select_one('a[onclick*="berater-spielerdaten"]')
empresario = empresario_tag.text.strip() if empresario_tag else "Empresário não encontrado"

# Clube atual
clube_tag = soup.select_one('a[title][href*="/startseite/verein/"]')
clube_atual = clube_tag['title'].strip() if clube_tag else "Clube não encontrado"

# Infos detalhadas
labels = soup.select('span.info-table__content.info-table__content--regular')
values = soup.select('span.info-table__content.info-table__content--bold')

inicio_clube = fim_contrato = ultima_renovacao = "Data não encontrada"
for label, value in zip(labels, values):
    if "No time desde:" in label.text:
        inicio_clube = value.text.strip()
    elif "Contrato até:" in label.text:
        fim_contrato = value.text.strip()
    elif "Última renovação de contrato:" in label.text:
        ultima_renovacao = value.text.strip()

# Mostrar no terminal
print("Nome completo:", player_full_name)
print("Apelido:", player_name)
print("Número:", shirt_number)
print("Posição:", position)
print("Nascimento e idade:", nascimento_idade)
print("Altura:", altura)
print("Nacionalidade:", nacionalidade)
print("Pé dominante:", dominant_foot)
print("Valor de mercado:", market_value)
print("Empresário:", empresario)
print("Clube atual:", clube_atual)
print("Início no clube atual:", inicio_clube)
print("Fim do contrato:", fim_contrato)
print("Última renovação:", ultima_renovacao)

# Dados para salvar
dados = [
    player_full_name, player_name, shirt_number, position,
    nascimento_idade, altura, nacionalidade, dominant_foot,
    market_value, empresario, clube_atual,
    inicio_clube, fim_contrato, ultima_renovacao
]

# Cabeçalhos
colunas = [
    "Nome Completo", "Apelido", "Número", "Posição",
    "Nascimento/Idade", "Altura", "Nacionalidade", "Pé Dominante",
    "Valor de Mercado", "Empresário", "Clube Atual",
    "Início no Clube", "Fim do Contrato", "Última Renovação"
]

# Criar ou carregar planilha
if os.path.exists(caminho_arquivo):
    wb = load_workbook(caminho_arquivo)
    ws = wb.active
else:
    wb = Workbook()
    ws = wb.active
    ws.append(colunas)  # adicionar cabeçalho na primeira linha

# Adicionar nova linha
ws.append(dados)

# Salvar
wb.save(caminho_arquivo)

print("✅ Dados adicionados à planilha com sucesso!")

Nome completo: Vinicius José Paixão de Oliveira Junior
Apelido: Vinicius Junior
Número: #7
Posição: Ponta Esquerda
Nascimento e idade: 12/07/2000 (25)
Altura: 1,76 m
Nacionalidade: Brasil
Pé dominante: Direito
Valor de mercado: €170.00 mi
Empresário: Roc Nation Sports
Clube atual: Real Madrid CF
Início no clube atual: 12/07/2018
Fim do contrato: 30/06/2027
Última renovação: 31/10/2023
✅ Dados adicionados à planilha com sucesso!
